In [24]:
import h2o
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

In [4]:
h2o.init()

H2O cluster uptime:,2 seconds 94 milliseconds
H2O cluster version:,3.3.0.99999
H2O cluster name:,anqi_fu
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [5]:
# Import walking gait data
gait = h2o.upload_file(path=h2o.locate("smalldata/glrm_test/subject01_walk1.csv"))
gait.describe()


Parse Progress: [##################################################] 100%
Uploaded pyfdd77725-6f54-4bce-a68f-ce96df250922 into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124
  chunk_type        chunk_name  count  count_percentage        size  \
0        C2S  2-Byte Fractions      1          0.806452      386  B   
1        C4S  4-Byte Fractions    123         99.193550     82.6 KB   

   size_percentage  
0         0.454064  
1        99.545940  
                            size  number_of_rows  number_of_chunks_per_column  \
0  172.16.2.30:54321     83.0 KB             151                            1   
1               mean     83.0 KB             151                            1   
2                min     83.0 KB             151                            1   
3                max     83.0 KB             151                            1   
4             stddev        0  B               0                            0   
5              total     83.0 KB             151      

,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.57324,107.79163,600.01556,1028.72498,-152.53879,399.40009,1034.90125,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.25998,550.69415,684.75494,-133.588,435.01773,627.49988,-170.86086,319.34705,444.43289,157.69023,376.44724,403.25574,102.54192,293.89526,368.28577,165.6133,341.2023,438.80435,-172.96315,365.72482,375.95242,-120.97533,304.46347,393.67728,-173.38676,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,58.7473,-74.0361,211.85968,69.02702,-69.30095,185.39088,48.22646,-140.09074,254.42706,36.65646,-88.18266,627.38177,1342.85413,-35.20795,511.80032,1427.44507,164.48206,551.72424,1453.58936,-249.89265,485.94559,1227.90454,216.05853,531.12134,1217.64429,-266.60974,424.32767,1129.09949,242.29697,460.03223,1123.80298,-304.79358,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.59163,502.82971,886.58942,-300.87775,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,199.80693,36.02512,-153.13719,228.04898,42.9706,-54.88903,675.97009,1562.28418,44.15432,683.50897,1567.27209,-94.07465,614.13971,1740.38916,-19.48838
maxs,2.5,641.12048,1091.43665,180.07411,661.46918,1078.02356,-80.37424,451.5914,1077.82214,55.36993,692.89514,788.11365,217.07512,766.44531,750.72375,136.25021,728.67242,672.42242,209.89946,674.53168,783.71576,-138.11635,773.97705,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.46951,821.66901,425.57733,-63.51654,750.77588,473.02411,-124.50978,812.50183,320.70309,108.161,935.61414,198.72064,86.30818,934.44775,

In [6]:
# Basic GLRM using quadratic loss and no regularization (PCA)
model = h2o.glrm(x=gait[1:], k=5, init="PlusPlus", loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=1000)
model.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1442537977605_2
     number_of_iterations  final_step_size  final_objective_value
0                      14         0.000067         1657548.270902


ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 1657548.27727
Misclassification Error (Categorical): 0.0


In [7]:
# Archetype to feature mapping (Y)
gait_y = model._model_json["output"]["archetypes"]
print gait_y

gait_y_np = np.array(model.archetypes())
feat_cols = range(2, gait_y_np.shape[1], 3)
num_feat = len(feat_cols)

# Plot first archetype on z-coordinate features
x_pts = range(1, num_feat+1)
y_pts = gait_y_np[0, feat_cols]
plt.plot(x_pts, y_pts, 'bo')
plt.plot(x_pts, [0]*num_feat, 'k--')   # Dashed line at zero

# Add feature labels to each point
feat_names = gait_y.col_header[1:]
for i in xrange(num_feat):
    plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
plt.title("First Archetype's Z-Coordinate Feature Weights")
plt.xlabel("Feature")
plt.ylabel("Archetypal Weight")
plt.show()

              rasisx       rasisy      rasisz      lasisx       lasisy  \
0  Arch1  628.247327  1055.611495  140.650702  621.018884  1039.838268   
1  Arch2  611.509896  1075.345359  175.957944  614.188526  1068.150010   
2  Arch3  621.127714  1083.086705  123.695645  637.187581  1068.188184   
3  Arch4  601.971078  1062.784561  118.634076  605.856936  1044.628608   
4  Arch5  608.676432  1053.346413  164.750653  629.614872  1045.860928   

       lasisz    vsacralx     vsacraly   vsacralz    ...       ltoemedz  \
0 -119.434111  427.968106  1045.151329  29.734451    ...     -35.227166   
1  -85.159931  413.974177  1070.701518  52.089606    ...     -42.379271   
2 -136.504413  430.105525  1071.204858  -7.741979    ...     -22.833740   
3 -141.831933  406.242208  1055.142357  -0.198568    ...     -31.293829   
4  -95.281246  420.974686  1051.081378  25.805586    ...     -26.705053   

     rtemplex     rtempley    rtemplez    ltemplex     ltempley   ltemplez  \
0  709.074293  1583.465972

In [8]:
# Projection into archetype space (X)
x_key = model._model_json["output"]["loading_key"]["name"]
gait_x = h2o.get_frame(x_key)
gait_x.show()

time_np = np.array(h2o.as_list(gait["Time"]))
gait_x_np = np.array(h2o.as_list(gait_x))

# Plot archetypes over time
lines = []
for i in xrange(gait_x_np.shape[1]):
    lines += plt.plot(time_np, gait_x_np[:,i], '-')
plt.title("Archetypes over Time")
plt.xlabel("Time")
plt.ylabel("Archetypal Projection")
plt.legend(lines, gait_x.col_names)
plt.show()

H2OFrame with 151 rows and 5 columns: 
      Arch1     Arch2     Arch3     Arch4     Arch5
0  0.004707 -0.165513 -0.457454  0.248734  1.389798
1  0.053337 -0.197641 -0.382857  0.174745  1.370407
2  0.109534 -0.240967 -0.304375  0.099150  1.352689
3  0.159667 -0.272671 -0.215140  0.021734  1.320475
4  0.203660 -0.292198 -0.111170 -0.058242  1.270260
5  0.246259 -0.312705 -0.000106 -0.135636  1.213073
6  0.271923 -0.306636  0.129866 -0.211796  1.125943
7  0.291559 -0.302752  0.268046 -0.280195  1.031242
8  0.302073 -0.300272  0.408739 -0.334713  0.930867
9  0.301437 -0.294492  0.551001 -0.375469  0.822975


In [9]:
# Reconstruct data from matrix product XY
pred = model.predict(gait)
pred.head()

H2OFrame with 10 rows and 123 columns: 
   reconstr_R.ASIS.X  reconstr_R.ASIS.Y  reconstr_R.ASIS.Z  reconstr_L.ASIS.X  \
0         613.275440        1059.813080         173.432247         635.518180   
1         614.171631        1058.332117         171.873684         636.481270   
2         615.440785        1057.064063         169.975203         637.823832   
3         616.765388        1056.332060         167.994368         639.158882   
4         618.333538        1056.490603         165.845816         640.664276   
5         620.143989        1057.208239         163.364085         642.397403   
6         621.827240        1058.876096         160.728593         643.878405   
7         623.549354        1060.994974         157.549574         645.439459   
8         625.145333        1063.472481         153.863296         646.912035   
9         626.438160        1066.136613         149.777778         648.092196   

   reconstr_L.ASIS.Y  reconstr_L.ASIS.Z  reconstr_V.Sacral.X  \
0   

In [ ]:
# Plot original and reconstructed L.Acromium.X over time
# lacro_np = np.array(gait["L.Acromium.X"])
# lacro_pred_np = np.array(pred["reconstr_L.Acromium.X"])
# line_orig = plt.plot(time_np, lacro_np, '-')
# line_imp = plt.plot(time_np, lacro_pred_np, '-')

# plt.title("Position of Left Acromium over Time")
# plt.xlabel("Time")
# plt.ylabel("X-Coordinate of Left Acromium")
# plt.legend([line_orig, line_imp], ["Original", "Imputed"])
# plt.show()

In [10]:
# Import walking gait data with missing values
gait_miss = h2o.upload_file(h2o.locate("smalldata/glrm_test/subject01_walk1_miss15.csv"))
gait_miss.describe()


Parse Progress: [##################################################] 100%
Uploaded py3a9a414b-9d5f-4015-a508-dc1bf27366f2 into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124
  chunk_type        chunk_name  count  count_percentage        size  \
0        C2S  2-Byte Fractions      1          0.806452      386  B   
1        C4S  4-Byte Fractions    123         99.193550     82.6 KB   

   size_percentage  
0         0.454064  
1        99.545940  
                            size  number_of_rows  number_of_chunks_per_column  \
0  172.16.2.30:54321     83.0 KB             151                            1   
1               mean     83.0 KB             151                            1   
2                min     83.0 KB             151                            1   
3                max     83.0 KB             151                            1   
4             stddev        0  B               0                            0   
5              total     83.0 KB             151      

,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.57324,107.79163,600.01556,1029.25427,-152.53879,399.40009,1034.90125,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.25998,550.69415,684.75494,-133.588,435.01773,627.49988,-170.86086,319.34705,445.08429,157.69023,376.44724,403.25574,102.54192,295.04089,368.36661,165.6133,341.2023,438.80435,-172.96315,365.72482,375.95242,-120.97533,304.46347,393.67728,-173.38676,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,60.01719,-74.0361,211.85968,70.39124,-69.30095,185.39088,48.22646,-140.09074,254.42706,36.65646,-88.18266,627.38177,1342.85413,-35.20795,511.80032,1427.44507,164.48206,551.72424,1453.58936,-249.89265,485.94559,1227.90454,216.05853,531.12134,1217.64429,-266.60974,424.32767,1129.49243,242.4415,460.20703,1123.80298,-304.79358,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.59163,503.10611,886.58942,-300.87775,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,202.74066,36.02512,-153.13719,228.04898,42.9706,-54.88903,675.97009,1562.28418,44.15432,683.50897,1567.27209,-94.07465,614.13971,1740.38916,-19.48838
maxs,2.5,641.12048,1091.26807,180.07411,661.46918,1078.02356,-80.82956,451.5914,1077.77014,55.29718,692.89514,788.11365,216.86717,766.44531,750.72375,136.25021,726.57483,672.42242,209.89946,674.53168,781.90839,-138.11635,772.35175,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.46951,821.66901,425.57733,-63.825,750.77588,472.92294,-125.0268,812.50183,320.70309,108.161,935.61414,197.97754,85.62352,934.44775,209

In [11]:
# Basic GLRM using quadratic loss and no regularization
model2 = h2o.glrm(x=gait_miss[1:], validation_frame=gait, k=15, init="PlusPlus", loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=500, min_step_size=1e-7)
model2.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1442537977605_5
     number_of_iterations  final_step_size  final_objective_value
0                     500     1.552504e-07         2378538.340822


ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 2378538.3492
Misclassification Error (Categorical): 0.0

ModelMetricsGLRM: glrm
** Reported on validation data. **

MSE: NaN
Sum of Squared Error (Numeric): 3144547.47005
Misclassification Error (Categorical): 0.0


In [18]:
# Impute missing data from X and Y
pred2 = model2.predict(gait_miss)
pred2.head()

H2OFrame with 10 rows and 123 columns: 
   reconstr_R.ASIS.X  reconstr_R.ASIS.Y  reconstr_R.ASIS.Z  reconstr_L.ASIS.X  \
0         613.666482        1067.829736         169.270331         635.488907   
1         612.491827        1064.721422         167.266903         634.767878   
2         613.792215        1064.616194         166.209626         636.134948   
3         612.597311        1061.764807         165.739180         635.080153   
4         612.280062        1058.981232         164.128795         634.575090   
5         613.370497        1060.040582         162.315595         635.836736   
6         634.049543        1056.430449         151.792310         656.989944   
7         634.705029        1058.348456         152.285632         658.159608   
8         635.771762        1056.591198         149.138900         658.710506   
9         636.789197        1058.622068         146.884716         660.093438   

   reconstr_L.ASIS.Y  reconstr_L.ASIS.Z  reconstr_V.Sacral.X  \
0   

In [26]:
# Plot original and imputed L.Acromium.X over time
lacro_np2 = np.array(h2o.as_list(gait["L.Acromium.X"]))
lacro_pred_np2 = np.array(h2o.as_list(pred2["reconstr_L.Acromium.X"]))
plt.plot(time_np, lacro_np2, 'b-')
plt.plot(time_np, lacro_pred_np2, 'g-')

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label='Imputed')
plt.legend([blue_patch, green_patch], ["Original", "Imputed"])
plt.show()